In [1]:
import cv2
import numpy as np
import dlib
import os
import glob
import random

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
predictor5_path = "./shape_predictor_5_face_landmarks.dat"
predictor68_path = "./shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "./dlib_face_recognition_resnet_model_v1.dat"

detector = dlib.get_frontal_face_detector()
sp5 = dlib.shape_predictor(predictor68_path)
sp68 = dlib.shape_predictor(predictor68_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [4]:
feats = []

In [5]:
def get_features(im):
    dets = detector(im, 1)
    if len(dets) == 0:
        return None  # no face detected :(
    for d in dets:
        # TODO: what if there are two faces in one pic --> seems ok
#         print(d)
        cur_feat = []
        x_l = d.left()
        x_r = d.right()
        y_t = d.top()
        y_b = d.bottom()


        width = x_r - x_l
        height = y_b - y_t
    #     cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 255, 0), im.shape[0]//150)
        shape = sp68(im, d)
        for i in range(0, 68):
            # TODO: normalize to 0 mean? How?
            feat_x = (shape.part(i).x - x_l) / width
            feat_y = (shape.part(i).y - y_t) / height
            cur_feat.append(feat_x)
            cur_feat.append(feat_y)

    #         cv2.circle(orig, (shape.part(i).x, shape.part(i).y), 1, (255, 0, 0), im.shape[0]//150)
    #     feats.append(cur_feat)
    return cur_feat

In [6]:
# plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))

In [90]:
from sklearn import svm, linear_model, calibration, ensemble

In [8]:
# X_train  = feats
# # y = [0, 1]

In [9]:
# clf = svm.LinearSVC() # TODO: consider non-linear kernels: svm.SVC()

In [13]:
# clf.fit(X_train, y)

In [14]:
# clf.predict([feats[1], feats[0]])

In [15]:
emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Emotion list

In [16]:
def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("google_dataset\\%s\\*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

In [17]:
def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            current_feats = get_features(gray)
            if current_feats:
                training_data.append(current_feats) #append image feats to training data list
                training_labels.append(emotions.index(emotion))
    
        for item in prediction: #repeat above process for prediction set
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            current_feats = get_features(gray)
            if current_feats:
                prediction_data.append(current_feats)
                prediction_labels.append(emotions.index(emotion))

    return training_data, training_labels, prediction_data, prediction_labels

In [18]:
sets_made = make_sets()

In [116]:
def run_recognizer(print_probabilities=False):
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
#     training_data, training_labels, prediction_data, prediction_labels = sets_made  # to speed up TEMP
    
    print ("training SVM classifier")
    print ("size of training set is:", len(training_labels), "images")
    print ("size of test set is:", len(prediction_labels), "images")
    
    X = training_data
    y = training_labels
    
    clf = svm.LinearSVC(C=5) # 64
    clf = calibration.CalibratedClassifierCV(svm.LinearSVC(C=5)) # 64

#     clf = svm.SVC(C=4, kernel='linear', probability=True, decision_function_shape='ovo') # 54
#     clf = linear_model.LogisticRegression(C=5)
#     clf = ensemble.RandomForestClassifier() # no use
#     clf = svm.SVC(C=0.8, kernel='rbf', gamma=20)  # it's worse

    clf.fit(X, y)
    print ("predicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0
    
    for image in training_data:
        pred = clf.predict([image]) #predict emotion
        if pred == training_labels[cnt]: #validate it
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    print("Training accuracy:", ((100*correct)/(correct + incorrect))) 

    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred = clf.predict([image]) #predict emotion
        if print_probabilities:
            print("+++++++++")
            print(prediction_labels)
            print(clf.predict_proba([image]))
        if pred == prediction_labels[cnt]: #validate it
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

In [117]:
run_recognizer(True)

training SVM classifier
size of training set is: 131 images
size of test set is: 30 images
predicting classification set
Training accuracy: 70.99236641221374
+++++++++
[0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7]
[[ 0.15750544  0.21862395  0.2678559   0.05776468  0.08282758  0.17142439
   0.04399807]]
+++++++++
[0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7]
[[ 0.2704979   0.21335393  0.1821911   0.05433077  0.10973443  0.14726418
   0.0226277 ]]
+++++++++
[0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7]
[[ 0.19080272  0.21977517  0.21057752  0.0723091   0.09982938  0.14773562
   0.0589705 ]]
+++++++++
[0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7]
[[ 0.16971337  0.08969121  0.28084524  0.04684246  0.08172078  0.22519149
   0.10599545]]
+++++++++
[0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 6

60.0

In [118]:
#Now run it
metascore = []
for i in range(0,12):
    correct = run_recognizer()
    print ("Test: got", correct, "percent correct!")
    metascore.append(correct)

print ("\n\nend score:", np.mean(metascore), "percent correct!")

training SVM classifier
size of training set is: 130 images
size of test set is: 31 images
predicting classification set
Training accuracy: 78.46153846153847
Test: got 61.29032258064516 percent correct!
training SVM classifier
size of training set is: 132 images
size of test set is: 29 images
predicting classification set
Training accuracy: 68.93939393939394
Test: got 65.51724137931035 percent correct!
training SVM classifier
size of training set is: 131 images
size of test set is: 30 images
predicting classification set
Training accuracy: 74.80916030534351
Test: got 60.0 percent correct!
training SVM classifier
size of training set is: 132 images
size of test set is: 30 images
predicting classification set
Training accuracy: 73.48484848484848
Test: got 66.66666666666667 percent correct!
training SVM classifier
size of training set is: 130 images
size of test set is: 31 images
predicting classification set
Training accuracy: 80.0
Test: got 61.29032258064516 percent correct!
training SV

AttributeError: 'LinearSVC' object has no attribute '_coef_'

In [105]:
import sklearn
sklearn.__version__

'0.19.0'